In [1]:
import os

import time

In [3]:
siteUrl = "http://www.boattrader.com"
siteUrl = "/".join([siteUrl, "search-results"])
siteUrl = "/".join([siteUrl, "NewOrUsed-used"])
siteUrl = "/".join([siteUrl, "Type-any"])
siteUrl = "/".join([siteUrl, "Category-all"])
siteUrl = "/".join([siteUrl, "Zip-68046"])
siteUrl = "/".join([siteUrl, "Radius-4000"])
siteUrl = "/".join([siteUrl, "sourcecode-nt"])
siteUrl = "/".join([siteUrl, "Sort-Length:DESC"])
print(siteUrl)

proxies = {'http': 'http://<<USERID>>:<<PASSWORD>>@23.19.51.245:40791/'}

http://www.boattrader.com/search-results/NewOrUsed-used/Type-any/Category-all/Zip-68046/Radius-4000/sourcecode-nt/Sort-Length:DESC


In [7]:
import requests
from bs4 import BeautifulSoup

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=siteUrl, proxies=proxies)
sitePageSoup = BeautifulSoup(sitePage.content, 'lxml')

#<a href="/.../Page-173,28" class="last">≫</a>
siteCss = 'div.results-footer > ul.pagination > li > a.last'
siteLinks = sitePageSoup.select(siteCss)
print(siteLinks)

max_page = 0
if len(siteLinks):
    max_page = int(siteLinks[0]["href"].split('/')[-1].split('-')[1].split(',')[0])
print(max_page)

[<a class="last" href="/search-results/NewOrUsed-used/Type-any/Category-all/Zip-68046/Radius-4000/sourcecode-nt/Sort-Length:DESC/Page-173,28">&amp;Gt;</a>]
173


In [9]:
from random import randint

i = randint(1, max_page)
tagUrl = "/".join([siteUrl, "Page-"+str(i)+",28"])
print(tagUrl)

tagPage = requests.get(url=tagUrl)
tagSoup = BeautifulSoup(tagPage.content, 'lxml')

http://www.boattrader.com/search-results/NewOrUsed-used/Type-any/Category-all/Zip-68046/Radius-4000/sourcecode-nt/Sort-Length:DESC/Page-49,28
Page 49 of 173 - Boats for sale near Papillion, NE - BoatTrader.com


In [14]:
import urllib.parse
import re

# Get the title of the page to prove we are progressing
tagTitle = tagSoup.select("head > title")
titleString = tagTitle[0].get_text().strip()
print(titleString)

# Get the links to the individual pages
pageLinks = tagSoup.select('a[href*=/listing/]')
for pageLink in pageLinks:
    pageUrl = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', pageLink['href']))
    print(pageUrl)

Page 49 of 173 - Boats for sale near Papillion, NE - BoatTrader.com
http://www.boattrader.com/listing/1983-j-boats-j-30-705568/
http://www.boattrader.com/listing/1974-creekmore-boats-cruiser-102944732/
http://www.boattrader.com/listing/1974-creekmore-boats-cruiser-102944732/
http://www.boattrader.com/listing/2002-j-c-mfg-inc-tritoon-306-102942129/
http://www.boattrader.com/listing/2002-j-c-mfg-inc-tritoon-306-102942129/
http://www.boattrader.com/listing/2002-bond-yachts-mc30-102938876/
http://www.boattrader.com/listing/2001-boston-whaler-28-conquest-102945635/
http://www.boattrader.com/listing/2011-boston-whaler-285-conquest-706487/
http://www.boattrader.com/listing/2006-boston-whaler-305-conquest-703781/
http://www.boattrader.com/listing/1999-campion-chase-910-102938901/
http://www.boattrader.com/listing/1999-campion-chase-910-102938901/
http://www.boattrader.com/listing/1982-carver-3007-aft-cabin-my-102960562/
http://www.boattrader.com/listing/1982-carver-3007-aft-cabin-my-102960562/

In [15]:
seen_urls_array = []
for i in range(1, max_page+1):
    tagUrl = "/".join([siteUrl, "Page-"+str(i)+",28"])
    #print(tagUrl)

    tagPage = requests.get(url=tagUrl)
    tagSoup = BeautifulSoup(tagPage.content, 'lxml')

    # Get the title of the page to prove we are progressing
    tagTitle = tagSoup.select("head > title")
    titleString = tagTitle[0].get_text().strip()
    #print(titleString)

    # Get the links to the individual pages
    pageLinks = tagSoup.select('a[href*=/listing/]')
    for pageLink in pageLinks:
        pageUrl = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', pageLink['href']))
        #print(pageUrl)
        if pageUrl not in seen_urls_array:
            seen_urls_array.append(pageUrl)
    
    # Destroy the tree when you're done working with it
    tagSoup.decompose()

# Show the number of individual page links you have
print(len(seen_urls_array))

4807


In [17]:
import pickle

with open('boattrader.pickle', 'wb') as handle:
    pickle.dump(seen_urls_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
import pickle

with open('boattrader.pickle', 'rb') as handle:
    seen_urls_array = pickle.load(handle)

In [ ]:
pageUrl = seen_urls_array[randint(0,len(seen_urls_array)-1)]
print(pageUrl)

page = requests.get(url=pageUrl)
soup = BeautifulSoup(page.content, 'lxml')

In [24]:
dealerCss = "#seller-name"
dealerLink = soup.select(dealerCss)
if len(dealerLink):
    dealerName = dealerLink[0].get_text().strip()
    print(dealerName)

    if dealerName == "Private Seller":
        model = soup.select("header > h1")
        if len(model):
            model = model[0].get_text().strip()
            model = re.sub(r'\s+', ' ', model)
        else:
            model = "Unknown"
        print(model)
        price = soup.select("div.bd-price-location > span")
        if len(price):
            price = price[0].get_text().strip()
        else:
            price = "Unknown"
        print(price)
        phone = soup.select("#call-now-details")
        if len(phone):
            phone = phone[0].get_text().strip()
        else:
            phone = ""
        print(phone)

Private Seller
2016 Yellowfin 26 Offshore
$128,000
(813) 226-7402


In [ ]:
for pageUrl in seen_urls_array:
    try:
        page = requests.get(url=pageUrl, proxies=proxies)
        soup = BeautifulSoup(page.content, 'lxml')

        dealerCss = "#seller-name"
        dealerLink = soup.select(dealerCss)
        if len(dealerLink):
            dealerName = dealerLink[0].get_text().strip()
            #print(dealerName)

            if dealerName == "Private Seller":
                model = soup.select("header > h1")
                if len(model):
                    model = model[0].get_text().strip()
                    model = re.sub(r'\s+', ' ', model)
                else:
                    model = "Unknown"
                #print(model)
                price = soup.select("div.bd-price-location > span")
                if len(price):
                    price = price[0].get_text().strip()
                else:
                    price = "Unknown"
                #print(price)
                phone = soup.select("#call-now-details")
                if len(phone):
                    phone = phone[0].get_text().strip()
                else:
                    phone = ""
                #print(phone)
                if len(phone):
                    bigFile = open('boattrader.txt', 'a', encoding='utf-8')
                    bigFile.write(model + '\t' + price + '\t' + phone + '\n')
                    bigFile.close()
    
        # Destroy the tree when you're done working with it
        soup.decompose()
        
    except Exception as e:
        print(str(e))